In [122]:
from typing import Generic, TypeVar, Dict, List, Optional
from abc import ABC, abstractmethod

In [123]:
V = TypeVar('V')
D = TypeVar('D') 

In [124]:
class Constraint(Generic[V, D], ABC):
    def __init__(self, variables: List[V]) -> None:
        self.variables = variables

    @abstractmethod
    def is_satisfied(self, assignment: Dict[V, D]) -> bool:
        ...

In [128]:
class Zoo_constraint(Constraint[str, str]):
    def __init__(self, animal1: str, animal2: str):
        super().__init__([animal1, animal2])
        self.animal1 = animal1
        self.animal2 = animal2
         
    def is_satisfied(self, assignment: Dict[str, int]):
        if self.animal1 == "Leao" and self.animal1 in assignment:
            return assignment[self.animal1] == 1
        if "Leao" in assignment and "Tigre" in assignment and self.animal1 == "Antilope" and self.animal1 in assignment:
                return assignment[self.animal1] != (assignment["Leao"] + 1) and assignment[self.animal1] != (assignment["Tigre"] + 1) and assignment[self.animal1] != (assignment["Tigre"] - 1) 
        if self.animal1 not in assignment or self.animal2 not in assignment:
            return True
        return assignment[self.animal1] != assignment[self.animal2]

In [129]:
class CSP(Generic[V, D]):
    def __init__(self, variables: List[V], domains: Dict[V, List[D]]):
        self.variables: List[V] = variables
        self.domains: Dict[V, List[D]] = domains 
        self.constraints: Dict[V, List[Constraint[V, D]]] = {}
        for variable in self.variables:
            self.constraints[variable] = []
            if variable not in self.domains:
                raise LookupError("Every variable should have a domain assigned to it")

    def add_constraint(self, constraint: Constraint[V, D]):
        for variable in constraint.variables:
            if variable not in self.variables:
                raise LookupError("Variable in constraint not in CSP")
            else:
                self.constraints[variable].append(constraint)

    def consistent(self, variable: V, assignment: Dict[V, D]):
        for constraint in self.constraints[variable]:
            if not constraint.is_satisfied(assignment):
                return False
        return True

    def mcv_algorithm(self, unassigned_animals):
        mcvariable = unassigned_animals[0]
        for variable in unassigned_animals:
            if len(self.constraints[mcvariable]) < len(self.constraints[variable]):
                mcvariable = variable
        return mcvariable
        

    def backtracking_search(self, assignment: Dict[V, D] = {}) -> Optional[Dict[V, D]]:
        if len(assignment) == len(self.variables):
            return assignment

        unassigned: List[V] = [v for v in self.variables if v not in assignment]

        start = self.mcv_algorithm(unassigned)
        for value in self.domains[start]:
            local_assignment = assignment.copy()
            local_assignment[start] = value
            if self.consistent(start, local_assignment):
                result: Optional[Dict[V, D]] = self.backtracking_search(local_assignment)
                if result is not None:
                    return result
        return None

In [130]:
class Animal_distribution():
    variables: List[str] = ["Tigre", "Hiena", "Javali" , "Leao", "Pavao", "Suricate", "Antilope"]
    domains: Dict[str, List[int]] = {}
    for variable in variables:
        domains[variable] = [1, 2, 3, 4]
    csp: CSP[str, str] = CSP(variables, domains)
    csp.add_constraint(Zoo_constraint("Tigre", "Pavao"))
    csp.add_constraint(Zoo_constraint("Tigre", "Suricate"))
    csp.add_constraint(Zoo_constraint("Tigre", "Javali"))
    csp.add_constraint(Zoo_constraint("Tigre", "Antilope"))
    csp.add_constraint(Zoo_constraint("Tigre", "Leao"))
    csp.add_constraint(Zoo_constraint("Hiena", "Leao"))
    csp.add_constraint(Zoo_constraint("Hiena", "Antilope"))
    csp.add_constraint(Zoo_constraint("Hiena", "Javali"))
    csp.add_constraint(Zoo_constraint("Hiena", "Pavao"))
    csp.add_constraint(Zoo_constraint("Hiena", "Suricate"))
    csp.add_constraint(Zoo_constraint("Javali", "Tigre"))
    csp.add_constraint(Zoo_constraint("Javali", "Hiena"))
    csp.add_constraint(Zoo_constraint("Leao", "Tigre"))
    csp.add_constraint(Zoo_constraint("Leao", "Pavao"))
    csp.add_constraint(Zoo_constraint("Pavao", "Leao"))
    csp.add_constraint(Zoo_constraint("Pavao", "Tigre"))
    csp.add_constraint(Zoo_constraint("Leao", "Antilope"))
    csp.add_constraint(Zoo_constraint("Antilope", "Leao"))
    csp.add_constraint(Zoo_constraint("Antilope", "Tigre"))
    csp.add_constraint(Zoo_constraint("Suricate", "Tigre"))
    csp.add_constraint(Zoo_constraint("Suricate", "Hiena"))
    solution: Optional[Dict[str, str]] = csp.backtracking_search()   
    if solution is not None:
        print("--------")
        for cage in range(1, len(domains[variable]) + 1):
            print("Cage", cage)
            for animal, zone in solution.items():
                    if zone == cage:
                        print(animal)
            print("--------")
    else:
        print("ERROR: Solution not found!")

--------
Cage 1
Leao
Javali
Suricate
--------
Cage 2
Tigre
Hiena
--------
Cage 3
Pavao
--------
Cage 4
Antilope
--------
